[View in Colaboratory](https://colab.research.google.com/github/mlindauer/lab_course_add/blob/master/Run_SMAC_Cheap_function_(Small_example).ipynb)

# Using SMAC to optimize an artificial black box function (Small example)

* Installation of SMAC and its dependencies
* Definition of cheap 1d function
* Plotting this function
* Using SMAC's fmin interface to optimize the function
* Plotting of how SMAC performed over time

## Installation of SMAC and its dependencies

In [0]:
!apt-get install swig -y
!pip install Cython
!pip install pyrfr==0.8.0 --no-cache --user
# hack to find pyrfr
import sys
sys.path.insert(0,"./.local/lib/python3.6/site-packages")

!pip install git+https://github.com/automl/SMAC3.git@development

## Define Function to be optimized

In [0]:
import math
import numpy as np

def test_func(x):
    # x is vector; here of length 1
    x = x[0]
    return math.cos(x) * x**2 + x 

## Plot Function

In [0]:
x_points = np.linspace(start=-5,stop=5,num=100) 
y_points = list(map(test_func,map(lambda x: [x], x_points)))

import matplotlib.pyplot as plt
plt.plot(x_points, y_points, 'k')

## Optimize with SMAC

In [0]:
from smac.facade.func_facade import fmin_smac

MAX_FUN = 30

x, cost, smac = fmin_smac(func=test_func,
                       x0=[-0], # default values
                       bounds=[(-5, 5)], # bounds of each x
                       maxfun=MAX_FUN, # maximal number of function evaluations 
                       rng=1234 # random seed
                       )

print("Best x: %f" %(x[0]))
print("Best y: %f" %(cost))

## Plot Found Solution

In [0]:
plt.plot(x_points, y_points, 'k', x, [cost], 'ro')

## How has SMAC performed over time?

In [0]:
import numpy as np
# get runhistory, all runs ever evaluated by smac
runhistory = smac.get_runhistory()

# extract x value and corresponding y value
x_smac = []
y_smac = []
for entry in runhistory.data: # iterate over data because it is an OrderedDict
  config_id = entry.config_id # look up config id
  config = runhistory.ids_config[config_id] # look up config
  y_ = runhistory.get_cost(config) # get cost
  x_ = config["x1"] # there is only one entry in our example
  x_smac.append(x_)
  y_smac.append(y_)
x_smac = np.array(x_smac)
y_smac = np.array(y_smac)


# Plotting
plt.plot(y_smac, 'o')
plt.xlabel('function evaluation')
plt.ylabel('function value')

# let's plot only the best function value at each time step
y_best = np.zeros(MAX_FUN)
for id in range(MAX_FUN):
  y_best[id] = np.min(y_smac[:id+1])
  
plt.step(y_best, 'k', where="post") # please note that we use step function!